### Train Models Using ML algorithms

In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

#### Importing Data

In [67]:
data = pd.read_csv("data/processed_data.csv")
print(data.shape)
data.head(2)

(1200, 64)


,Unnamed: 0,Age,DistanceFromHome,EmpEducationLevel,EmpEnvironmentSatisfaction,EmpHourlyRate,EmpJobInvolvement,EmpJobLevel,EmpJobSatisfaction,NumCompaniesWorked,...,maritalstatus__Single,edubackground__Human Resources,edubackground__Life Sciences,edubackground__Marketing,edubackground__Medical,edubackground__Other,edubackground__Technical Degree,gender__Female,gender__Male,emp_number
0,0,32,10,3,4,55,3,2,4,1,...,1,0,0,1,0,0,0,0,1,1001000
1,1,47,14,4,4,42,3,2,1,2,...,1,0,0,1,0,0,0,0,1,1001006


### Selecting X and y values


In [68]:
X = data.drop(['PerformanceRating'],axis=1)
y = data.PerformanceRating

### Splitting the Data to  Train and Test Data

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

### since classes are not balanced, making the data as balanced 

In [70]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [71]:
#!pip install imblearn
#!conda install -c conda-forge/label/gcc7 imbalanced-learn
#!pip install -U imbalanced-learn

### Scale the data before training

In [72]:
X_train_final = StandardScaler().fit_transform(X_train_smote)

In [73]:
X_test_final = StandardScaler().fit_transform(X_test)

### Training model using Random Forest

In [75]:

RF_model = RandomForestClassifier(max_depth=11,n_estimators=15,max_features=63,criterion='gini',random_state=11)
RF_model.fit(X_train_smote,y_train_smote)
#y_predict = model.predict(X_test)

RandomForestClassifier(max_depth=11, max_features=63, n_estimators=15,
                       random_state=11)

### Save this trained model in pickle format for later use

In [76]:
model_file = open("rf_model.pkl","wb")
pickle.dump(RF_model,model_file)

### Model Prediction

In [77]:
y_predict = RF_model.predict(X_test)

### Evaluation of Models

In [95]:
def eval_models(y_test,y_predict):
    #print("Showing Accuracy \n")
    print("The accuracy of this Random Forest Model is ",round(accuracy_score(y_test,y_predict) * 100,2))
    
def model_class_report(y_test,y_predict):
    print("Showing Classification report \n")
    print(classification_report(y_test,y_predict))
    print("Showing crosstab \n")
    print(pd.crosstab(y_test,y_predict))

In [96]:
eval_models(y_test,y_predict)
model_class_report(y_test,y_predict)

The accuracy of this Random Forest Model is  91.25
Showing Classification report 

              precision    recall  f1-score   support

           2       0.84      0.84      0.84        38
           3       0.97      0.94      0.95       179
           4       0.66      0.83      0.73        23

    accuracy                           0.91       240
   macro avg       0.82      0.87      0.84       240
weighted avg       0.92      0.91      0.92       240

Showing crosstab 

col_0               2    3   4
PerformanceRating             
2                  32    3   3
3                   4  168   7
4                   2    2  19


**Here we achieved an accuracy of 91.2 % with the Random Forest Model**

### Hyper Parameter Tuning using Random Search Method

#### Define parameters

In [87]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [50, 100, 150, 200, 250, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [88]:
Rand_Forest = RandomForestClassifier()
RF_Tuned = RandomizedSearchCV(estimator = Rand_Forest, param_distributions = random_grid,
                              n_iter = 25, cv = 3, verbose=2, random_state=42)
RF_Tuned.fit(X_train_smote,y_train_smote)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=250; total time=   0.9s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=250; total time=   1.1s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=250; total time=   1.1s


C:\Users\sajana\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=250; total time=   1.0s


C:\Users\sajana\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=250; total time=   1.1s


...

C:\Users\sajana\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   1.2s


C:\Users\sajana\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   1.1s


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=25,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300]},
                   random_state=42, verbose=2)

#### Prediction with Tuned Random Forest Model

In [89]:
y_predict_tuned = RF_Tuned.predict(X_test)

#### Evaluation

In [97]:
eval_models(y_test,y_predict_tuned)
model_class_report(y_test,y_predict_tuned)

The accuracy of this Random Forest Model is  94.58
Showing Classification report 

              precision    recall  f1-score   support

           2       0.95      0.92      0.93        38
           3       0.96      0.97      0.96       179
           4       0.83      0.83      0.83        23

    accuracy                           0.95       240
   macro avg       0.91      0.90      0.91       240
weighted avg       0.95      0.95      0.95       240

Showing crosstab 

col_0               2    3   4
PerformanceRating             
2                  35    3   0
3                   2  173   4
4                   0    4  19


**After the hyper parameter tuning, Random forest model accuracy is improved from 91.2% to 94.5% which is a reliable accuracy for model prediction**